In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import time
import numpy as np

In [74]:
import pydrake
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import SceneGraph
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.optimization import CalcGridPointsOptions, Toppra
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import RevoluteJoint
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import TrajectorySource
from pydrake.trajectories import PiecewisePolynomial
from pydrake.all import Variable
from pydrake.all import MultibodyPositionToGeometryPose, ConnectMeshcatVisualizer, Role, Sphere
from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
    Role, Sphere,
    Iris, IrisOptions, MakeIrisObstacles, Variable
)
from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)

import pydrake.symbolic as sym

from meshcat import Visualizer

# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

In [10]:
simple_collision = True
# gripper_welded = True

vis = Visualizer(zmq_url=zmq_url)
vis.delete()
display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False)

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [19]:
seed_points = np.array([[0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]])    # out of shelf 1


from pydrake.all import InverseKinematics
cspace_seed_points = []
hand_body = plant.GetBodyByName("iiwa_link_ee_kuka")
hand_frame = hand_body.body_frame()
q0 = plant.GetPositions(plant_context)
for i in range(seed_points.shape[0]):
    ik = InverseKinematics(plant, plant_context)
    ik.AddMinimumDistanceConstraint(0.001, 0.01)
    ik.AddPositionConstraint(hand_frame, [0, 0, 0], plant.world_frame(), seed_points[i,:], seed_points[i,:])
    ik.prog().SetInitialGuess(ik.q(), q0)
    result = Solve(ik.prog())
    if not result.is_success():
        print(seed_points[i,:])
#         assert result.is_success()
    cspace_seed_points.append(result.GetSolution(ik.q()))
cspace_seed_points = np.array(cspace_seed_points)
print(cspace_seed_points)

# seed_points = np.array([[0.0, 0.0, 0.0, 0.0,0.0, 0.0]])    # out of shelf 1

# traj = PiecewisePolynomial.FirstOrderHold(np.array([0, 1]), np.array([seed_points[4], seed_points[1]]).T)
# visualize_trajectory(traj)

[ 0.    -2.016  1.975]
[-1.  -2.   0.5]
[ 0.3 -0.8  0.5]
[ 0.25 -1.6  -0.25]
[ 0.07 -1.8  -0.2 ]
[-0.1 -2.  -0.3]
[[ 3.98592487e-04  3.51284882e-06  8.23286705e-06]
 [-1.42226165e+00  1.12769674e-05  1.95171828e-05]
 [ 3.22695147e-01  3.51284882e-06  8.23286705e-06]
 [ 2.67448200e-01  3.51284882e-06  8.23286705e-06]
 [ 7.48112845e-02  3.51284882e-06  8.23286705e-06]
 [-1.05905251e-01  3.51284882e-06  8.23286705e-06]]


In [12]:
def MakeFromSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return HPolyhedron(query, geom, expressed_in)

In [53]:
def FindClosestCollision(sym_plant, sym_context, frameA, frameB, setA, setB, E, Gt, ht):
    prog = MathematicalProgram()
    #mismatch in shape of t and E.
    t = prog.NewContinuousVariables(Gt.shape[1], "t")

    p_AA = prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)

    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), t)

    sym_plant.SetPositions(sym_context, t)
    X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, frameA)
    X_WB = sym_plant.EvalBodyPoseInWorld(sym_context, frameB)
    print(X_WB)
    # Add +0 pending https://github.com/RobotLocomotion/drake/issues/15216
    p_WA = X_WA.multiply(p_AA+0)
    p_WB = X_WB.multiply(p_BB+0)
    prog.AddConstraint(eq(p_WA, p_WB))
    result = snopt.Solve(prog)


In [54]:
def IrisInConfigSpace(plant, plant_context, sample, iris_options):
    #reimplement IrisInConfigurationSpace method in Python so I can tweak grow ellipse
    nq = plant.num_positions()
    P = HPolyhedron(plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits())
    kEpsilonEllipsoid = 1e-2
    E = Hyperellipsoid.MakeHypersphere(kEpsilonEllipsoid, sample)
    query_object = plant.get_geometry_query_input_port().Eval(plant_context)
    inspector = query_object.inspector()

    #Skipping IrisConvex set maker steps? Should be IrisConvexSetMaker
    pairs = inspector.GetCollisionCandidates()
    N = len(pairs)
    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromSceneGraph(query_object, geom, inspector.GetFrameId(geom)) for geom in geom_ids}

    #Symbolic plants and bodies
    sym_plant = plant.ToSymbolic()
    sym_context = sym_plant.CreateDefaultContext()
    sym_bodies = {geom:sym_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}

    #preallocated
    Gt = np.zeros((P.A().shape[0]+2*N, nq))
    ht = np.zeros(P.A().shape[0]+2*N)

    best_volume = E.Volume()
    iteration = 0

    #ignoring solver logic?
    i = 0
#     while True:
    while i < 1:
        num_constraints = 2*nq
        # Find separating hyperplanes


        for (geomA, geomB) in pairs:
            print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
            collision  = FindClosestCollision(sym_plant, sym_context, sym_bodies[geomA], sym_bodies[geomB],
                                              sets[geomA], sets[geomB], E,
                                              Gt[:num_constraints, :], ht[:num_constraints])
            #only do one
            break
        i+=1


In [55]:
iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 10
iris_options.enable_ibex = False

regions = []
for i in range(seed_points.shape[0]):
    start_time = time.time()
#     hpoly = IrisInConfigurationSpace(plant, plant_context, seed_points[i,:], iris_options)
    hpoly = IrisInConfigSpace(plant, plant_context, seed_points[i,:], iris_options)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    regions.append(hpoly)


geomA=wsg::left_collision, geomB=shelves::shelf_lower
RigidTransform_[Expression](
  R=RotationMatrix_[Expression]([
    [<Expression "1">, <Expression "0">, <Expression "0">],
    [<Expression "0">, <Expression "1">, <Expression "0">],
    [<Expression "0">, <Expression "0">, <Expression "1">],
  ]),
  p=[<Expression "0.80000000000000004">, <Expression "0">, <Expression "0.40000000000000002">],
)


NameError: name 'snopt' is not defined

In [83]:
#unpack a symbolic geometry
query_object = plant.get_geometry_query_input_port().Eval(plant_context)
inspector = query_object.inspector()
pairs = inspector.GetCollisionCandidates()
sym_plant = plant.ToSymbolic()
sym_context = sym_plant.CreateDefaultContext()
geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
sym_bodies = {geom:sym_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
geomA = iter(pairs).__next__()[0]
frameA = sym_bodies[geomA]
prog = MathematicalProgram()
#dangerous since I am adding the q variable to the program
q = prog.NewContinuousVariables(3, "q")
sym_plant.SetPositions(sym_context, q)
X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, frameA)
p_AA = prog.NewContinuousVariables(3, "p_AA")
p_WA = X_WA.multiply(p_AA+0)
for i in range(p_WA.shape[0]):
    p_WA[i] = p_WA[i].Expand()

In [97]:
# print(p_WA[0])
tmp = p_WA[0]
print(sym.DecomposeAffineExpressions(tmp), tmp.Variables())

TypeError: DecomposeAffineExpressions(): incompatible function arguments. The following argument types are supported:
    1. (expressions: numpy.ndarray[object[m, 1]], vars: numpy.ndarray[object[m, 1]]) -> Tuple[numpy.ndarray[numpy.float64[m, n]], numpy.ndarray[numpy.float64[m, 1]]]
    2. (v: numpy.ndarray[object[m, 1]]) -> Tuple[numpy.ndarray[numpy.float64[m, n]], numpy.ndarray[numpy.float64[m, 1]], numpy.ndarray[object[m, 1]]]

Invoked with: <Expression "(-7.3478807948841384e-18 - 1.2246467991470917e-16 * p_AA(0) - 2.6630217472597643e-43 * p_AA(1) + 8.0041065145432615e-23 * p_AA(2) - 7.657137397852263e-16 * (p_AA(0) * sin(q(0))) - 1.2246467991470917e-16 * (p_AA(0) * sin(q(0)) * sin(q(1))) - 3.7029323991872667e-22 * (p_AA(0) * sin(q(0)) * sin(q(1)) * sin(q(2))) - 6.5358489648738163e-07 * (p_AA(0) * sin(q(0)) * sin(q(1)) * cos(q(2))) + 2.8621074166753517e-53 * (p_AA(0) * sin(q(0)) * sin(q(2))) + 6.5358489648738163e-07 * (p_AA(0) * sin(q(0)) * sin(q(2)) * cos(q(1))) - 3.702932399187266

In [80]:
t = prog.NewContinuousVariables(q.shape[0], "t")

In [122]:
import sympy
sympy_expr = sympy.parse_expr(tmp.to_string())
print(sympy_expr.variables)

AttributeError: 'Add' object has no attribute 'variables'

In [117]:

def extract_cos_tmp(expr):
    expr_string = expr.to_string()
    string2 = expr_string.replace("sin(q(0))", "(1-t(0)**2)/(1+t(0)**2)")
    return string2
one_sub = extract_cos_tmp(tmp)


-3.7029323991872667e-22*p_AA(0)*sin(q(0))*sin(q(1))*sin(q(2)) - 6.5358489648738163e-7*p_AA(0)*sin(q(0))*sin(q(1))*cos(q(2)) - 1.2246467991470917e-16*p_AA(0)*sin(q(0))*sin(q(1)) + 6.5358489648738163e-7*p_AA(0)*sin(q(0))*sin(q(2))*cos(q(1)) + 2.8621074166753517e-53*p_AA(0)*sin(q(0))*sin(q(2)) - 3.7029323991872667e-22*p_AA(0)*sin(q(0))*cos(q(1))*cos(q(2)) + 6.1288543328915224e-38*p_AA(0)*sin(q(0))*cos(q(2)) - 7.657137397852263e-16*p_AA(0)*sin(q(0)) - 6.5358489648738163e-7*p_AA(0)*sin(q(1))*sin(q(2))*cos(q(0)) + 2.721778469103341e-53*p_AA(0)*sin(q(1))*sin(q(2)) + 3.7029323991872667e-22*p_AA(0)*sin(q(1))*cos(q(0))*cos(q(2)) + 6.1288543328915224e-38*p_AA(0)*sin(q(1))*cos(q(2)) + 1.1483866719614894e-47*p_AA(0)*sin(q(1)) - 3.7029323991872667e-22*p_AA(0)*sin(q(2))*cos(q(0))*cos(q(1)) - 3.1775775529434021e-69*p_AA(0)*sin(q(2))*cos(q(0)) - 6.1288543328915224e-38*p_AA(0)*sin(q(2))*cos(q(1)) + 3.5776342708441896e-54*p_AA(0)*sin(q(2)) - 6.5358489648738163e-7*p_AA(0)*cos(q(0))*cos(q(1))*cos(q(2)) - 1